<h1 style="color:#FF0000">Set-up</h1>


### Import libraries
- Imports libraries needed for this notebook

In [30]:
pip install chart_studio

In [31]:
import pandas as pd
from datetime import datetime as dt
import urllib.request
import json
import yfinance as yf
import time
from tqdm import tqdm
from requests.exceptions import HTTPError, RequestException
import plotly.express as px

In [32]:
# Import and setup for plotly in Colab
import chart_studio
import chart_studio.plotly as py # for plotting graphs
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px

# Enable displaying pandas data frames as interactive tables by default
from google.colab import data_table # so tables don't suck
data_table.enable_dataframe_formatter()

- Pick your start and end dates, <br>
- whether you want the peer data or the company data,<br>
- and what threshold you want for percentage presence over time of each company's stock within an index <br>
- choose your file <br>
- and your index <br>
- download_lag is the pause in seconds between evaluating each ticker, i recommend 0.5 to prevent yahoo throttling your access

In [33]:
start ='2014-01-01' #@param {type:string} ['2014-01-01','2019-12-01']
end = 'today' #@param {type:string} ['2019-11-01','today']
benchmark= False #@param {type:'boolean'}
threshold=90 #@param {type:'integer'}
chosen_ticker_file = 'S&P500tickers.csv' #@param {type:string} ['S&P500tickers.csv','FTSE100.csv']
fin_index= 'SP500' #@param {type:string} ['SP500','FTSE100']
download_lag= 0.5 #@param {type: "slider", min: 0, max: 2, step: 0.01}


if end == 'today':
  end = None
# when slicing with None it goes right up to the end of the dataframe

#create time variable to name dataset
if start == '2014-01-01':
  if end==None:
    time_range='all'
  elif end=='2019-11-01':
    time_range='before'
else:
    time_range='after'

#create data variable to name dataset
if benchmark==True:
    datatype='peerGroup'
else:
    datatype='ticker'

<h1 style="color:#FF0000">Tickers list</h1>

### Create tickers list function
- Takes a CSV for an index fund with a list of dates, each with a corresponding list of tickers that were in an index on that date
- Also takes a threshold percentage time (out of 100) that a stock has to have been on the index (default 90)
- Returns a list of tickers which have been on that index a specified percentage of the time between 2014 and 2025

In [34]:
# code I borrowed from: https://medium.com/@rganesh0203/different-ways-to-count-duplicates-in-list-python-in-worst-to-best-complexities-8a23c3ebd9c4
# this code just counts how many times an element occurs in a list and puts that in a dictionary with key as element and value as number of occurences
def count_duplicates_dict(lst):
        duplicates = {}
        for item in lst:
            if item in duplicates:
                duplicates[item] += 1
            else:
                duplicates[item] = 1
        return {key: value for key, value in duplicates.items() if value > 1}

In [35]:
def tickers_func(file, threshold=90):
    strongest=[]      #creates an empty list
    df=pd.read_csv(file)      #read csv file

    df['date']=pd.to_datetime(df['date'])      #set date column as datetime object
    df=df[df['date']>'2014-01-01']      #we only want data pre 2014
    df_list=list(df['tickers'])      #create a list, each element is a string that contains all the stock tickers for that date
    total=len(df_list)      #gets total number of timestamp rows

    for index,value in enumerate(df_list):      #iterates through df_list
        strongest.extend(value.split(','))      #splits the strings of tickers into lists of strings of single tickers and adds all those lists into one big list
    dict_best_tickers=count_duplicates_dict(strongest)      #uses the count duplicates function to count how many times a ticker shows up in the list

    for i in dict_best_tickers:      #iterates through dictionary and replaces count of occurences with percentage of occurences
        dict_best_tickers[i]=round(100*dict_best_tickers[i]/total,2)
    sorted_dict = dict(sorted(dict_best_tickers.items(), key=lambda item: item[1],reverse=True))      #sorts by percentage of occurences

    final_dict=dict()      #create empty dictionary

    for i in sorted_dict:      #iterate through dictionary, adding values which exceed the occurence threshold to the empty dictionary
        if sorted_dict[i]>threshold:
            final_dict[i]=sorted_dict[i]

    tickers=list(final_dict.keys())      #gets the keys i.e. tickers of the dictionary and puts them in a list

    return tickers      #exports the list


### S&P 500 tickers list

### These will need to be changed if the index is changed (has been done manually)

In [36]:
tickers=tickers_func(chosen_ticker_file, threshold=threshold)      #calls the function above

In [37]:
# used claude anthropic to sort failing tickers into tickers to change and tickers to remove
change_dict= {
        'FB': 'META',
        'ANTM': 'ELV',   #updated tickers
        'BLL': 'BALL',
        'DISCA': 'WBD',
        'CTL': 'LUMN',
        'MYL': 'VTRS',
        'FISV': 'FI',
        'COG': 'CTRA',
        'BF.B':'BF-B',
        'BRK.B':'BRK-B',



        }
remove_list =[
        'A',
        'NLSN',
        'CERN',
        'XLNX',
        'KSU',
        'ALXN',  #comapanies that have gone private
        'VAR',
        'TIF',
        'NBL',
        'ETFC',
        'ADS',
        'AGN',
        'FLIR',
        'GPS',
        'MRO',
        'PXD',
        'PKI',
        'CTXS',
        'PBCT',
        'ABC',
        'GOOG',
        'LB'
        ]

In [38]:
for _index, _value in change_dict.items():
  tickers[tickers.index(_index)]=_value
for _value in remove_list:
  tickers.remove(_value)

### FTSE 100 stock data

<h1 style="color:#FF0000">Create metadata tables</h1>

In [39]:
list_metrics=list(yf.Ticker('AAPL').sustainability.index) #creates list of possible information we can get from sustainability attribute
columns=['Symbol','Name_short','Name_long','Zip_code','City','State','Country','Website','Industry','Sector','Full-time_Employees']
#metrics we've chosen to take from yf.Ticker(ticker string i.e. "AAPL").info
columns.extend(list_metrics) #add on sustainability metrics
metadata=pd.DataFrame(columns=columns)
# creates a dataframe called metadata with each row as a ticker and its associated metadata
# using lots of try/except pairs to handle non-US companies not having state or zip code



for ticker in tqdm(tickers): #tqdm makes a progress bar, iterate over tickers list, choose tickers to choose index

    print(ticker)

    time.sleep(download_lag) #prevents request speed throttling for having too fast of a rate


    tick=yf.Ticker(ticker) #initialises Ticker class
    stock=tick.info #gets info attr
    esg = tick.sustainability #gets sustainability attr


    try:
        has_zip=stock['zip'] #zip and state handling
        has_state=stock['state']
    except:
        has_zip=None
        has_state=None


    try:
        has_employees=stock['fullTimeEmployees'] #missing employee data handling
    except:
        has_employees=None


    stock_row=[stock['symbol'],stock['shortName'],stock['longName'],has_zip,stock['city'],has_state,stock['country'],stock['website'],stock['industry'],stock['sector'],has_employees]
    stock_row.extend([esg.loc[metric,'esgScores'] for metric in list_metrics]) # creates stock_row which contains all the metadata for that ticker
    stock_row=pd.DataFrame([stock_row],columns=columns) #turns stock_row into dataframe
    metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe

  0%|          | 0/348 [00:00<?, ?it/s]

AAPL


<ipython-input-39-53559c05036c>:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe
  0%|          | 1/348 [00:00<03:27,  1.67it/s]

ABBV


  1%|          | 2/348 [00:01<03:32,  1.63it/s]

...

<ipython-input-39-53559c05036c>:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe
 99%|█████████▉| 346/348 [03:35<00:01,  1.64it/s]

JWN


<ipython-input-39-53559c05036c>:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe
100%|█████████▉| 347/348 [03:35<00:00,  1.64it/s]

HP


<ipython-input-39-53559c05036c>:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metadata=pd.concat([metadata, stock_row], ignore_index=True) #adds stock_row onto metadata master dataframe
100%|██████████| 348/348 [03:36<00:00,  1.61it/s]


# Cleaning data

In [40]:
metadata

,Symbol,Name_short,Name_long,Zip_code,City,State,Country,Website,Industry,Sector,...,smallArms,furLeather,gambling,gmo,militaryContract,nuclear,pesticides,palmOil,coal,tobacco
0,AAPL,Apple Inc.,Apple Inc.,95014,Cupertino,CA,United States,https://www.apple.com,Consumer Electronics,Technology,...,False,False,False,False,False,False,False,False,False,False
1,ABBV,AbbVie Inc.,AbbVie Inc.,60064-6400,North Chicago,IL,United States,https://www.abbvie.com,Drug Manufacturers - General,Healthcare,...,False,False,False,False,False,False,False,False,False,False
2,ABT,Abbott Laboratories,Abbott Laboratories,60064-6400,North Chicago,IL,United States,https://www.abbott.com,Medical Devices,Healthcare,...,False,False,False,False,False,False,False,False,False,False
3,ACN,Accenture plc,Accenture plc,None,Dublin,None,Ireland,https://www.accenture.com,Information Technology Services,Technology,...,False,False,False,False,False,False,False,False,False,False
4,ADBE,Adobe Inc.,Adobe Inc.,95110-2704,San Jose,CA,United States,https://www.adobe.com,Software - Infrastructure,Technology,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,LUMN,"Lumen Technologies, Inc.","Lumen Technologies, Inc.",71203,Monroe,LA,United States,https://www.lumen.com,Telecom Services,Communication Services,...,False,False,False,False,False,False,False,False,False,False
344,UAA,"Under Armour, Inc.","Under Armour, Inc.",21230,Baltimore,MD,United States,https://www.underarmour.com,Apparel Manufacturing,Consumer Cyclical,...,False,False,False,False,False,False,False,False,False,False
345,HOG,"Harley-Davidson, Inc.","Harley-Davidson, Inc.",53208,Milwaukee,WI,United States,https://www.harley-davidson.com,Recreational Vehicles,Consumer Cyclical,...,False,False,False,False,False,False,False,False,False,False
346,JWN,"Nordstrom, Inc.","Nordstrom, Inc.",98101,Seattle,WA,United States,https://press.nordstrom.com,Department Stores,Consumer Cyclical,...,False,False,False,False,False,False,False,False,False,False


In [41]:
#no cleaning yet

<h1 style="color:#FF0000">Upload to GBQ</h1>

In [42]:
from google.colab import auth #authenticates google colab
auth.authenticate_user()

In [43]:
#@title access BigQuery project
project_id = 'lw-esg-project'

# Package used for interfacing w/ BigQuery from Python
from google.cloud import bigquery

# Create BigQuery client
bq_client = bigquery.Client(project = project_id)

I viciously raided this google colab notebook for code: https://colab.research.google.com/drive/1hSI1BXyCyj7viRpp1GFZqkU1qtBUd0g1?authuser=0 (produced for google by Alok Pattani I believe...)

Select the dataset, only one atm
Select output_table_id (chooses which df to upload to a table)
Select replace or append
The way I've written this that should be all you need to do

# DO NOT - I REPEAT - DO NOT: USE THIS WITHOUT CHECKING WITH AT LEAST ONE OTHER PERSON, this code can and will rewrite any work we do on bigquery (which is why it's commented out)

In [45]:
#@title upload dataframe to BigQuery Table
output_dataset_id = 'yfinance'

output_table_id = 'metadata'

replace_or_append_output = 'replace' #@param{type:'string'} ['replace', 'append']


#don't forget to change this!!!!!!!!!!!!!!!!!!!!!!!!!!!!
output_table=metadata #choose dataframe from dictionary created previously using selected string output_dataset_id



if output_table_id=='metadata':
    output_table_id=output_table_id
else:
    output_table_id = f"{output_table_id}_{time_range}_{datatype}"
output_table_id = f"{output_table_id}_{fin_index}_thresh{threshold}"




# Combine project and dataset
project_dataset = f"{bq_client.project}.{output_dataset_id}"

# Combine project, dataset, and table
project_dataset_table = f"{project_dataset}.{output_table_id}"

# Check to make sure output dataset exists, create it if not
try:
  bq_client.get_dataset(output_dataset_id)
  print(f"BigQuery dataset {project_dataset} exists\n")

except:
  print(f"BigQuery dataset {project_dataset} doesn't exist, so creating it\n")
  dataset = bq_client.create_dataset(bigquery.Dataset(project_dataset))

job_config = bigquery.LoadJobConfig()

# Modify job config depending on if we want to replace or append to table
if(replace_or_append_output == 'replace'):
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
else:
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

dataset_ref = bq_client.dataset(output_dataset_id)
table_ref = dataset_ref.table(output_table_id)


# Use client functionality to load BigQuery table from Pandas data frame
bq_client.load_table_from_dataframe(
  dataframe = output_table,
  destination = table_ref,
  job_config = job_config
  ).result()

print((f"{output_table_id} ({replace_or_append_output}) to "
  f"BigQuery table {project_dataset_table}\n"))


BigQuery dataset lw-esg-project.yfinance exists



/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: peerEsgScorePerformance, peerGovernancePerformance, peerSocialPerformance, peerEnvironmentPerformance, peerHighestControversyPerformance, environmentPercentile, socialPercentile, governancePercentile.
  warnings.warn(


metadata_SP500_thresh90 (replace) to BigQuery table lw-esg-project.yfinance.metadata_SP500_thresh90

